In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy import stats

# import funcs as funcs
from lib.import_funcs import *
%load_ext autoreload
%autoreload 2

In [2]:
# path = 'C:/Users/cleli/Dropbox (Dropbox_2021)/Clelia/Work/' # for Windows
path = '/Users/cleliacorridori/Dropbox (Dropbox_2021)/Clelia/Work/' # for Mac

2/04/2022 

Clelia Corridori


Program to extract the following matrices:
- normalized with Scran
- normalized with PsiNorm
- Raw data

and information:
- present cell list
- present gene list
- indices for the initial and final columns of a precise time step matrices (contained in the whole gene expression matrix)

and the matrices _only for the 24 important genes_ for:
- normalized with Scran
- normalized with PsiNorm
- Raw data



In [3]:
df = pd.read_csv(path+"input_data/ScranNorm_matx_w00.csv", header=1, sep=",")
df

,1,1.1,AAACCCAAGCTGTCCG,AAACCCAAGCTTAAGA,AAACCCAAGGAGCTGT,AAACCCACACTACCGG,AAACCCACAGTTACCA,AAACCCACATCATCCC,AAACCCAGTAGCGTCC,AAACCCAGTGACCGTC,...,TTTGTTGAGTTTCTTC,TTTGTTGCAGTAGAAT,TTTGTTGCATCGAAGG,TTTGTTGCATCTCATT,TTTGTTGGTAACAGGC,TTTGTTGGTATGTCAC,TTTGTTGTCACAACCA,TTTGTTGTCACTGAAC,TTTGTTGTCAGCAATC,TTTGTTGTCCCGTTCA
0,2,Sox17,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3,Mrpl15,1.569892,0.842834,1.295267,2.072526,2.645378,1.530503,1.957817,0.000000,...,0.971488,0.000000,2.228101,1.624867,1.402606,1.327581,0.781272,1.736623,1.648932,0.000000
2,4,Lypla1,0.988712,0.000000,1.295267,0.849244,0.000000,0.344644,0.972015,0.000000,...,1.546820,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.048241,1.259716
3,5,Tcea1,0.000000,0.000000,0.000000,0.000000,0.000000,1.232163,0.972015,1.731219,...,0.971488,1.179186,0.000000,0.000000,0.000000,0.000000,0.781272,0.000000,1.648932,0.000000
4,6,Rgs20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13828,13830,ENSMUSG00000079834,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13829,13831,ENSMUSG00000079800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13830,13832,LOC102638047,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13831,13833,ENSMUSG00000095041,0.988712,1.757326,1.966595,0.000000,0.000000,1.056134,1.547528,0.000000,...,0.000000,0.000000,1.156002,0.000000,2.100162,0.000000,1.285278,2.169544,0.000000,1.628414


In [4]:
matx = np.squeeze(np.array(df))[:,1:]

# the present cells (=barcodes)
barcodes = np.array(df.columns)[2:]
# print('List of cell names : ', barcodes)
# print("Number of barcodes", len(barcodes))
np.savetxt(path+'/general_info/all_cells_list.csv', barcodes, fmt='%s')

# the present genes (=features)
genes = np.array(matx[:,0])
# print("present genes: ", genes[:5])
np.savetxt(path+'/general_info/all_genes_list.csv', genes, fmt='%s')


#gene expression matrix
matx = matx[:,1:]
# print(matx[:5,:5])

print("number of genes: ", np.shape(matx)[0])
print("\nnumber of cells: ", np.shape(matx)[1])

number of genes:  13833

number of cells:  9894


## link time and barcodes

In [4]:
path_short = '/Users/cleliacorridori/Dropbox (Dropbox_2021)/Clelia/send_wt/'
time_tab = np.array(pd.read_csv(path_short+'RC9_seurat_hashID.csv',
                                 header=0, sep=',', quotechar='"'))

In [5]:
print(np.shape(time_tab))
print(time_tab[0,:])

(16194, 2)
['AAACCCAAGCTGTCCG' '48h']


In [6]:
time_points= np.unique(time_tab[:,1]) #label

#divide in subset the barcodes, the groups correspond to different time points
tab_00 = time_tab[time_tab[:,1]=="00h"][:,0]
tab_06 = time_tab[time_tab[:,1]=="06h"][:,0]
tab_12 = time_tab[time_tab[:,1]=="12h"][:,0]
tab_24 = time_tab[time_tab[:,1]=="24h"][:,0]
tab_48 = time_tab[time_tab[:,1]=="48h"][:,0]
tab_D = time_tab[time_tab[:,1]=="Doublet"][:,0]
tab_N = time_tab[time_tab[:,1]=="Negative"][:,0]

In [7]:
# indices of the barcodes in the different time points
barcodes_00, idx_bar00, idx_times00 = np.intersect1d(barcodes, tab_00, return_indices=True)
print("00h", len(idx_bar00), len(barcodes), len(tab_00))

barcodes_06, idx_bar06, idx_times06 = np.intersect1d(barcodes, tab_06, return_indices=True)
print("06h", len(idx_bar06), len(barcodes), len(tab_06))

barcodes_12, idx_bar12, idx_times12 = np.intersect1d(barcodes, tab_12, return_indices=True)
print("12h", len(idx_bar12), len(barcodes), len(tab_12))

barcodes_24, idx_bar24, idx_times24 = np.intersect1d(barcodes, tab_24, return_indices=True)
print("24h", len(idx_bar24), len(barcodes), len(tab_24))

barcodes_48, idx_bar48, idx_times48 = np.intersect1d(barcodes, tab_48, return_indices=True)
print("48h", len(idx_bar48), len(barcodes), len(tab_48))

barcodes_D,idx_barD, idx_timesD= np.intersect1d(barcodes, tab_D,return_indices=True)
print("D", len(idx_barD),len(barcodes), len(tab_N))

barcodes_N, idx_barN, idx_timesN = np.intersect1d(barcodes, tab_N, return_indices=True)
print("N", len(idx_barN),len(barcodes), len(tab_N))

NameError: name 'barcodes' is not defined

In [9]:
# np.savetxt(path+'/general_info/cells_times/00_cells_indices.csv', idx_bar00, fmt='%s')
# np.savetxt(path+'/general_info/cells_times/06_cells_indices.csv', idx_bar06, fmt='%s')
# np.savetxt(path+'/general_info/cells_times/12_cells_indices.csv', idx_bar12, fmt='%s')
# np.savetxt(path+'/general_info/cells_times/24_cells_indices.csv', idx_bar24, fmt='%s')
# np.savetxt(path+'/general_info/cells_times/48_cells_indices.csv', idx_bar48, fmt='%s')

# to save all the indices for the different time steps
times_idx_barcodes_matx = [idx_bar00, idx_bar06, idx_bar12, idx_bar24, idx_bar48]

np.savez(path+"general_info/cells_times/times_cells_indices.npz", idx_00=idx_bar00, idx_06=idx_bar06, idx_12=idx_bar12, idx_24=idx_bar24, idx_48=idx_bar48)

## Saving the gene expression DATA for all the normalizations

In [3]:

#scran
matx_00, matx_06, matx_12, matx_24, matx_48, matx_clean = GE_matrix_out(path, "ScranNorm_matx")

#PsiNorm
matx_00, matx_06, matx_12, matx_24, matx_48, matx_clean = GE_matrix_out(path, "PsiNorm_matx", index=False)

#Raw
matx_00, matx_06, matx_12, matx_24, matx_48, matx_clean = GE_matrix_out(path, "raw_matx", index=False)

In [4]:
print(np.shape(matx_00), np.shape(matx_06), np.shape(matx_12),  np.shape(matx_24), np.shape(matx_48))

(13833, 2025) (13833, 2532) (13833, 2025) (13833, 1765) (13833, 1547)


### Find important genes

In [7]:
#DATA only of the 24 important genese
#scran
matx_impGenes_ScranNorm = GE_matrix_out_MostImp(path, "ScranNorm_matx")

#PsiNorm
matx_impGenes_PsiNorm = GE_matrix_out_MostImp(path, "PsiNorm_matx")

#Raw
matx_impGenes_raw_matx = GE_matrix_out_MostImp(path, "raw_matx")

ScranNorm_matx: matrix saved
PsiNorm_matx: matrix saved
raw_matx: matrix saved
